In [4]:
import pandas as pd
import geopandas as gpd

import os
from os import listdir
from os.path import isfile, join
import pathlib

#Pandas display options to show more data when DataFrames are shown in the notebook (defaults are quite limited)
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300
pd.options.display.precision = 5

#pygeos and pyogrio can speed up geopandas but can also cause package dependency conflicts - disable if conflicts occur on your system.
#gpd.options.use_pygeos = True
import pyogrio #Only works if (engine = "pyogrio") is used in geopandas .to_file() code. 

In [5]:
#Important! this code will not run unless the "onsset_replication" data folder is downloaded and included in the file structure as in the instructions

years = [2020, 2030, 2040, 2050, 2060]
scenarios = ["baseline", "moderate_development", "sustainable_development"]

mled_scenario_directory = r"onsset_replication/mled/onsset_clusters"
mled_scenario_filenames = list(pathlib.Path(mled_scenario_directory).glob('*.gpkg'))
scenario_dictionary = dict(zip(scenarios, mled_scenario_filenames))

output_directory_for_onsset = r"mled_processed_input_files"

In [7]:
#Read in the clusters file data including all of the related geospatial data etc. Excludes the geometry of the cluster shapes. 
#This is then merged with the MLED demand data below.
clusters_geo = pd.read_csv("onsset_replication/clusters/zambia_onsset_input_calibrated.csv")

In [11]:
#Note: This can take some time. Will also be slower if pyogrio is not used. Will likely need more than 8GB of RAM.

#Important! this code will not run unless the "onsset_replication" data folder is downloaded and included in the file structure as in the instructions

for mled_scenario in scenarios:

    gdf = gpd.read_file(os.path.join(scenario_dictionary[mled_scenario]), engine = "pyogrio")
    
    print("\n")
    print("MLED Scenario processing into OnSSET input files for: '", mled_scenario,"'...")
    print("Taken from file at: ", os.path.join(scenario_dictionary[mled_scenario]))
    
    for year in years:
        gdf["tot_dem_per_capita_"+str(year)] = gdf["tot_dem_"+str(year)] / gdf["population_"+str(year)]
    for year in years:
        print("Total Demand--"+str(year)+": ",(gdf["tot_dem_"+str(year)].sum()/10**9).round(2), " TWh")
    for year in years:
        print("Per Capita Demand--"+str(year)+": ", (gdf["tot_dem_per_capita_"+str(year)].mean()).round(2), " kWh/cap/year")
    for year in years:
        print("Population--"+str(year)+": "+str((gdf["population_"+str(year)].sum()).round(0)))
    
    print("Urban Pop Percentage--2020: ", (gdf.query("isurban_2020 == 1").population_2020.sum()/gdf.population_2020.sum()).round(4))
    print("Urban Pop Percentage--2030: ", (gdf.query("isurban_future_2030 == 1").population_2030.sum()/gdf.population_2030.sum()).round(4))
    print("Urban Pop Percentage--2040: ", (gdf.query("isurban_future_2040 == 1").population_2040.sum()/gdf.population_2040.sum()).round(4))
    print("Urban Pop Percentage--2050: ", (gdf.query("isurban_future_2050 == 1").population_2050.sum()/gdf.population_2050.sum()).round(4))
    print("Urban Pop Percentage--2060: ", (gdf.query("isurban_future_2060 == 1").population_2060.sum()/gdf.population_2060.sum()).round(4))
          
    gdf.drop(columns = "geometry", inplace = True)
    
    df = pd.DataFrame(gdf[gdf.columns[-11:]]) #add only newly calculated columns to a pandas dataframe for selected export
    
    df["id"] = gdf["id"].astype("int")
    df[['isurban', 'isurban_2020', 'isurban_future_2030', 'isurban_future_2040', 'isurban_future_2050', 'isurban_future_2060']] = gdf[['isurban', 'isurban_2020', 'isurban_future_2030', 'isurban_future_2040', 'isurban_future_2050', 'isurban_future_2060']]
    df[["population_2020","population_2030","population_2040","population_2050","population_2060"]] = gdf[["population_2020","population_2030","population_2040","population_2050","population_2060"]]
    df[["gdp_capita_2020","gdp_capita_2030","gdp_capita_2040","gdp_capita_2050","gdp_capita_2060"]] = gdf[["gdp_capita_2020","gdp_capita_2030","gdp_capita_2040","gdp_capita_2050","gdp_capita_2060"]]
    df.isurban = df.isurban.replace(1,2)
    
    mled = df
    onsset = pd.merge(clusters_geo, mled, on = "id", how = "right")
    
    #Calculation and conditioning of variables used in OnSSET for custom demand levels
    onsset["IsUrban"] = onsset["isurban"]
    onsset["PerCapitaDemand"] = 0
    onsset["GDP"] = 0
    onsset["ResidentialDemandTierCustom"] = onsset["tot_dem_per_capita_2030"]
    
    onsset["PerCapitaDemand2020"] = onsset["tot_dem_per_capita_2020"]
    onsset["PerCapitaDemand2030"] = onsset["tot_dem_per_capita_2030"]
    onsset["PerCapitaDemand2040"] = onsset["tot_dem_per_capita_2040"]
    onsset["PerCapitaDemand2050"] = onsset["tot_dem_per_capita_2050"]
    onsset["PerCapitaDemand2060"] = onsset["tot_dem_per_capita_2060"]
    
    onsset["NumPeoplePerHH"] = 5
    onsset.NumPeoplePerHH.mask(onsset.isurban > 0, 4.7, inplace = True)
    onsset.NumPeoplePerHH.mask(onsset.isurban == 0, 5.2, inplace = True)
    
    onsset.Pop = onsset.population_2020
    onsset["Pop2020"] = onsset.population_2020
    onsset["Pop2030"] = onsset.population_2030
    onsset["Pop2040"] = onsset.population_2040
    onsset["Pop2050"] = onsset.population_2050
    onsset["Pop2060"] = onsset.population_2060
    
    onsset.to_csv(os.path.join(output_directory_for_onsset, "zambia_onsset_input_mled_demands_"+ str(mled_scenario) + ".csv"), index = False) 
    



MLED Scenario processing into OnSSET input files for: ' baseline '...
Taken from file at:  onsset_replication\mled\onsset_clusters\1_zambia_onsset_clusters_with_mled_loads_ssp2_rcp60_0.998_0.248_0.1_tot_lat_d.gpkg
Total Demand--2020:  13.12  TWh
Total Demand--2030:  40.47  TWh
Total Demand--2040:  46.55  TWh
Total Demand--2050:  58.27  TWh
Total Demand--2060:  75.08  TWh
Per Capita Demand--2020:  1340.96  kWh/cap/year
Per Capita Demand--2030:  2060.43  kWh/cap/year
Per Capita Demand--2040:  2172.49  kWh/cap/year
Per Capita Demand--2050:  2336.74  kWh/cap/year
Per Capita Demand--2060:  2636.78  kWh/cap/year
Population--2020: 19255577.0
Population--2030: 24179801.0
Population--2040: 29271172.0
Population--2050: 34015019.0
Population--2060: 38279939.0
Urban Pop Percentage--2020:  0.4187
Urban Pop Percentage--2030:  0.4762
Urban Pop Percentage--2040:  0.53
Urban Pop Percentage--2050:  0.5787
Urban Pop Percentage--2060:  0.6221


MLED Scenario processing into OnSSET input files for: ' mod